In [18]:
import pandas as pd
from bcb import sgs, Expectativas
from datetime import datetime, timedelta
import jinja2
import numpy as np

In [19]:
# função para fazer a formatação das DataFrames

def formata_df(df: pd.DataFrame, format_data_rf: str, valor: str):
    
    # Data and DataRef to datetime
    df['DataReferencia'] = pd.to_datetime(df['DataReferencia'], format=format_data_rf)

    
    df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')
    df['Data']= df['Data'].dt.strftime('%d/%m/%Y')

    # table pivot - table sorting by header - formating header
    df_pivot = df.pivot_table(index='Data', columns='DataReferencia', values=valor)
    df_pivot = df_pivot.sort_index(axis=1)
    df_pivot.columns = df_pivot.columns.strftime('%Y')

    # Data formating
    df_pivot = df_pivot.sort_index(key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

    # changing dtypes=float
    df_pivot = df_pivot.astype(float)    
    
    return df_pivot

In [20]:
# EndPoints
em = Expectativas()

ep_top5_anual = em.get_endpoint('ExpectativasMercadoTop5Anuais')
ep_m_anual = em.get_endpoint('ExpectativasMercadoAnuais')
ep_12meses = em.get_endpoint('ExpectativasMercadoInflacao12Meses')
ep_24meses = em.get_endpoint('ExpectativasMercadoInflacao24Meses')


EntitySets:
  ExpectativasMercadoTop5Anuais
  ExpectativaMercadoMensais
  ExpectativasMercadoInflacao24Meses
  ExpectativasMercadoInflacao12Meses
  ExpectativasMercadoSelic
  ExpectativasMercadoTop5Selic
  ExpectativasMercadoTop5Mensais
  ExpectativasMercadoTrimestrais
  ExpectativasMercadoAnuais


In [21]:
# IPCA top5 anual MedioPrazo
ipca_top5_anual = ep_top5_anual.query().filter(ep_top5_anual.Indicador == 'IPCA', ep_top5_anual.tipoCalculo == 'M', ep_top5_anual.Data >= '2023-01-01').select(ep_top5_anual.Indicador, ep_top5_anual.Data, ep_top5_anual.Mediana, ep_top5_anual.DataReferencia).collect()
ipca_top5_anual_pivot = formata_df(ipca_top5_anual, '%Y', 'Mediana')

In [22]:
# IPCA Media anual
ipca_media_anual = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2024-02-29', ep_m_anual.baseCalculo == '0').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Media, ep_m_anual.baseCalculo).collect()
ipca_media_anual_pivot = formata_df(ipca_media_anual, '%Y', 'Media')

# IPCA Mediana anual
ipca_mediana_anual = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2024-02-29', ep_m_anual.baseCalculo == '0').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Mediana, ep_m_anual.baseCalculo).collect()
ipca_mediana_anual_pivot = formata_df(ipca_mediana_anual, '%Y', 'Mediana')




In [23]:
ipca_ult5dias = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01', ep_m_anual.baseCalculo == '1').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Mediana, ep_m_anual.baseCalculo).collect()
ipca_ult5dias_pivot = formata_df(ipca_ult5dias, '%Y', "Mediana")

In [24]:
# Expec inflacao IPCA ->>> ta uma confusao por que nao usa o metodo formata_df()


# 12 meses -> mediana
ipca_12meses = ep_12meses.query().filter(ep_12meses.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01', ep_12meses.baseCalculo == '0').select(ep_12meses.Indicador, ep_12meses.Data, ep_12meses.Mediana, ep_24meses.Suavizada).collect()

# Formatando data
ipca_12meses['Data'] = pd.to_datetime(ipca_12meses['Data'], format='%Y-%m-%d')
ipca_12meses['Data']= ipca_12meses['Data'].dt.strftime('%d/%m/%Y')

# Formatando header
ipca_12meses_pivot = ipca_12meses.pivot_table(index='Data', columns='Suavizada', values='Mediana')

# Formatando tabela
ipca_12meses_pivot.applymap(lambda x: '{:.4f}'.format(x) if not pd.isnull(x) else np.nan)
ipca_12meses_pivot.insert(loc=0, column='12 Meses', value='')


# 13 a 24 meses -> mediana
ipca_24meses = ep_24meses.query().filter(ep_24meses.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01', ep_24meses.baseCalculo == '0').select(ep_24meses.Indicador, ep_24meses.Data, ep_24meses.Mediana, ep_24meses.Suavizada).collect()

# Formatando data
ipca_24meses['Data'] = pd.to_datetime(ipca_24meses['Data'], format='%Y-%m-%d')
ipca_24meses['Data']= ipca_24meses['Data'].dt.strftime('%d/%m/%Y')

# Formatando header
ipca_24meses_pivot = ipca_24meses.pivot_table(index='Data', columns='Suavizada', values='Mediana')

# Formatando tabela
ipca_24meses_pivot.applymap(lambda x: '{:.4f}'.format(x) if not pd.isnull(x) else np.nan)
ipca_24meses_pivot.insert(loc=0, column='13-24 Meses', value='')

# Concatenando tabelas
ipca_concat = pd.concat([ipca_12meses_pivot, ipca_24meses_pivot], axis=1)
ipca_concat = ipca_concat.rename(columns={'N': 'Não Suav', 'S': 'Suav'})
ipca_concat.columns.name=''

# Formatando Data
ipca_concat = ipca_concat.sort_index(key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))



,12 Meses,Não Suav,Suav,13-24 Meses,Não Suav,Suav
Data,,,,,,
02/01/2023,,5.2931,5.3053,,3.8453,3.7972
03/01/2023,,5.2928,5.3009,,3.8402,3.7903
04/01/2023,,5.2802,5.2985,,3.8402,3.7870
05/01/2023,,5.2931,5.3186,,3.8506,3.7979
06/01/2023,,5.3027,5.3592,,3.8606,3.7959
...,...,...,...,...,...,...
22/03/2024,,3.4013,3.4520,,3.5969,3.6017
25/03/2024,,3.4011,3.4499,,3.6000,3.6010
26/03/2024,,3.4011,3.4534,,3.6000,3.6010


In [25]:
# to excel

with pd.ExcelWriter(r'C:\Users\rm170600\Documents\Python\Projeto_API_SGS\Expec_Inflacao.xlsx') as writer:
   ipca_mediana_anual_pivot.to_excel(writer, sheet_name='IPCA Mediana Anual')
   ipca_media_anual_pivot.to_excel(writer, sheet_name='IPCA Media Anual')
   ipca_ult5dias_pivot.to_excel(writer, sheet_name='IPCA ult 5 dias')
   ipca_top5_anual_pivot.to_excel(writer, sheet_name='IPCA Top5 Anual')
   ipca_concat.to_excel(writer, sheet_name='Excp Inflação Suav')